In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
import fiona
from sklearn.impute import KNNImputer
from sklearn.preprocessing import LabelEncoder

pd.options.display.max_columns = None 

# set working directory
os.chdir('/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/Data')

In [2]:
# Load Data
combined_gdf = gpd.read_file('combined_gdf.geojson')

# Data Cleaning


In [3]:
print(combined_gdf.info())
print(combined_gdf.shape)

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 163137 entries, 0 to 163136
Data columns (total 46 columns):
 #   Column                          Non-Null Count   Dtype         
---  ------                          --------------   -----         
 0   Operator                        163137 non-null  object        
 1   Fluid                           160923 non-null  object        
 2   Material                        157338 non-null  object        
 3   Diam_in                         105790 non-null  float64       
 4   Status                          129183 non-null  object        
 5   Length_ft                       163137 non-null  float64       
 6   SHAPE_Length                    163137 non-null  float64       
 7   LOCATION_ID                     163113 non-null  float64       
 8   FLOWLINEID                      163023 non-null  float64       
 9   STARTLOCATIONID                 162850 non-null  float64       
 10  FLOWLINEACTION                  153765 non-null 

In [4]:
combined_gdf.describe()

,Diam_in,Length_ft,SHAPE_Length,LOCATION_ID,FLOWLINEID,STARTLOCATIONID,RECEIVE_DATE,OPERATOR_NUM,ENDLAT,ENDLONG,STARTLAT,STARTLONG,MAXOPPRESSURE,CONSTRUCTDATE,flowline_match_distance_m,unique_id,spill_trkg_num,spill_Long,spill_Lat,spill_matched_crudeoil_idx,spill_match_distance_m,risk
count,105790.000000,163137.000000,163137.000000,163113.000000,163023.000000,162850.000000,163137,163137.000000,163137.000000,163137.000000,163137.000000,163137.000000,32133.000000,163137,1.631370e+05,163137.000000,8.340000e+02,834.000000,834.000000,834.000000,834.000000,163137.000000
mean,3.770868,1808.009112,551.693904,410642.274215,465036.729511,352029.526816,2019-07-06 00:19:24.510000,73102.943262,39.862606,-105.270050,39.862655,-105.270919,361.652196,2003-11-27 23:24:37.298000,4.275996e+03,80457.715092,4.027870e+08,-106.105282,39.905904,81149.762590,808.739982,0.005112
min,0.000000,0.000000,0.000100,159601.000000,455152.000000,159652.000000,2018-05-07 10:15:51.367000,710.000000,37.010428,-109.045542,36.993609,-109.041450,0.000000,1900-01-11 00:00:00,0.000000e+00,45.000000,4.015243e+08,-108.979884,37.014053,44.000000,0.007369,0.000000
25%,2.000000,400.270000,122.158324,332226.000000,461359.000000,319558.000000,2019-01-29 12:05:38.490000,16700.000000,39.660524,-105.040682,39.665030,-105.040724,60.000000,1994-08-19 00:00:00,1.191985e+02,40662.000000,4.021619e+08,-108.111489,39.545005,25618.250000,7.943790,0.000000
50%,3.000000,947.950000,288.886951,431329.000000,463298.000000,329848.000000,2019-06-25 11:13:49.143000,100322.000000,40.280500,-104.619964,40.279660,-104.618080,150.000000,2006-06-11 00:00:00,6.713086e+02,80605.000000,4.029491e+08,-104.978823,40.074190,75383.500000,33.362815,0.000000
75%,4.000000,1784.370000,543.675787,461159.000000,467057.000000,410073.000000,2019-08-28 12:18:37.953000,100322.000000,40.415251,-104.498300,40.416175,-104.497470,245.000000,2010-11-24 00:00:00,2.436944e+03,120393.000000,4.033769e+08,-104.542132,40.351272,143903.000000,192.371522,0.000000
max,278.000000,152023.900000,46342.379790,483552.000000,484080.000000,484366.000000,2023-11-17 11:11:43.017000,200077.000000,40.992630,-102.045863,40.993770,-102.046467,4700.000000,2020-03-09 00:00:00,1.167436e+06,159144.000000,4.040505e+08,-102.083057,40.997350,162825.000000,57440.586845,1.000000
std,2.665691,5641.898762,1721.296096,61666.012262,6094.558871,52684.101115,NaN,39716.469435,1.032040,1.624048,1.032135,1.625511,708.876001,NaN,9.730529e+03,45930.857996,6.991696e+05,1.885338,0.689533,56637.108869,3409.229930,0.071317


### Get line age from construction date

In [5]:
# Verify CONSTRUCTDATE is datetime type; convert if necessary
combined_gdf['CONSTRUCTDATE'] = pd.to_datetime(combined_gdf['CONSTRUCTDATE'])

# Calculate line_age
# Today's date
today = pd.Timestamp.now()

# Calculate the difference in years
combined_gdf['line_age_yr'] = (today - combined_gdf['CONSTRUCTDATE']).dt.days / 365.25

# Display the updated GeoDataFrame to verify 'line_age' column
print(combined_gdf[['CONSTRUCTDATE', 'line_age_yr']])

       CONSTRUCTDATE  line_age_yr
0         1972-08-07    52.780287
1         1992-12-28    32.388775
2         2006-03-09    19.195072
3         1987-03-20    38.165640
4         1997-03-09    28.194387
...              ...          ...
163132    2005-06-01    19.964408
163133    2004-11-01    20.544832
163134    2005-05-26    19.980835
163135    2009-01-21    16.323066
163136    2005-04-28    20.057495

[163137 rows x 2 columns]


In [6]:
# Print the list of column names
column_names = combined_gdf.columns.tolist()

print(column_names)

['Operator', 'Fluid', 'Material', 'Diam_in', 'Status', 'Length_ft', 'SHAPE_Length', 'LOCATION_ID', 'FLOWLINEID', 'STARTLOCATIONID', 'FLOWLINEACTION', 'ENTIRELINEREMOVED', 'ACTIONDESCRIPTION', 'RECEIVE_DATE', 'OPERATOR_NUM', 'COMPANY_NAME', 'LOCATIONTYPE', 'ENDLAT', 'ENDLONG', 'STARTLAT', 'STARTLONG', 'PIPEMATERIAL', 'BEDDINGMATERIAL', 'TYPEOFFLUIDTRANS', 'MAXOPPRESSURE', 'CONSTRUCTDATE', 'flowline_match_distance_m', 'unique_id', 'spill_trkg_num', 'spill_Operator Name', 'spill_facility_type', 'spill_Spill_Desc', 'spill_Spill Type', 'spill_Root Cause', 'spill_Preventative Measure', 'spill_Root Cause Type', 'spill_Detailed Root Cause Type', 'spill_Long', 'spill_Lat', 'spill_facility_status', 'spill_Metallic?', 'spill_incident_date', 'spill_matched_crudeoil_idx', 'spill_match_distance_m', 'risk', 'geometry', 'line_age_yr']


In [7]:
mapping = {
    'KINDER MORGAN CO2 CO LP': 'KINDER MORGAN CO2 CO LLC',
    'BEEMAN OIL & GAS INC': 'BEEMAN OIL & GAS LLC',
}
combined_gdf['Operator'] = combined_gdf['Operator'].replace(mapping)

combined_gdf.rename(columns={'OPERATOR_NUM': 'operator_number'}, inplace=True)
combined_gdf.rename(columns={'Operator': 'operator_name'}, inplace=True)

# Extract unique OPERATOR_NUM values
unique_operator_nums = combined_gdf['operator_number'].unique()

# Create a DataFrame to map OPERATOR_NUM to Operator (assuming the column name for operator names is 'Operator')
combined_gdf_operator_mapping = combined_gdf[['operator_number', 'operator_name']].drop_duplicates().reset_index(drop=True)

# Display the mapping
print(combined_gdf_operator_mapping)

     operator_number                        operator_name
0              10110  GREAT WESTERN OPERATING COMPANY LLC
1              69175                       PDC ENERGY INC
2              47120      KERR MCGEE OIL & GAS ONSHORE LP
3             100322                     NOBLE ENERGY INC
4              10459             EXTRACTION OIL & GAS INC
..               ...                                  ...
125            11001                  BROWN OIL & GAS LLC
126            70350                     PILAND RESOURCES
127            16830                CHOLLA PRODUCTION LLC
128             6720        ROBERT L BAYLESS PRODUCER LLC
129            10348           MYSTIQUE RESOURCES COMPANY

[130 rows x 2 columns]


In [8]:
# assume your GeoDataFrame is called `combined_gdf`
# build a rename mapping for all columns starting with "spill_" except the one you want to keep
rename_dict = {
    col: col[len("spill_"):]
    for col in combined_gdf.columns
    if col.startswith("spill_") and col != "spill_match_distance_m"
}

print("Will rename these columns:")
for old, new in rename_dict.items():
    print(f"  {old} → {new}")

# apply the renaming
combined_gdf = combined_gdf.rename(columns=rename_dict)

# check the result
print("\nNew column list:")
print(combined_gdf.columns.tolist())

Will rename these columns:
  spill_trkg_num → trkg_num
  spill_Operator Name → Operator Name
  spill_facility_type → facility_type
  spill_Spill_Desc → Spill_Desc
  spill_Spill Type → Spill Type
  spill_Root Cause → Root Cause
  spill_Preventative Measure → Preventative Measure
  spill_Root Cause Type → Root Cause Type
  spill_Detailed Root Cause Type → Detailed Root Cause Type
  spill_Long → Long
  spill_Lat → Lat
  spill_facility_status → facility_status
  spill_Metallic? → Metallic?
  spill_incident_date → incident_date
  spill_matched_crudeoil_idx → matched_crudeoil_idx

New column list:
['operator_name', 'Fluid', 'Material', 'Diam_in', 'Status', 'Length_ft', 'SHAPE_Length', 'LOCATION_ID', 'FLOWLINEID', 'STARTLOCATIONID', 'FLOWLINEACTION', 'ENTIRELINEREMOVED', 'ACTIONDESCRIPTION', 'RECEIVE_DATE', 'operator_number', 'COMPANY_NAME', 'LOCATIONTYPE', 'ENDLAT', 'ENDLONG', 'STARTLAT', 'STARTLONG', 'PIPEMATERIAL', 'BEDDINGMATERIAL', 'TYPEOFFLUIDTRANS', 'MAXOPPRESSURE', 'CONSTRUCTDATE', 'f

In [9]:
# List of columns to be removed
columns_to_remove = [
    'trkg_num', 'Operator Name', 'facility_type', 'Spill_Desc', 'Spill Type', 'Root Cause', 'Preventative Measure', 'Detailed Root Cause Type', 
    'Long', 'Lat', 'facility_status', 'Metallic?', 'ACTIONDESCRIPTION', 'BEDDINGMATERIAL', 'COMPANY_NAME', 
    'ENDLAT', 'ENDLONG', 'ENTIRELINEREMOVED', 'PIPEMATERIAL', 'RECEIVE_DATE', 'STARTLAT', 'STARTLOCATIONID', 'STARTLONG', 'TYPEOFFLUIDTRANS', 'operator_name', 'SHAPE_Length', 
    'matched_crudeoil_idx', 'flowline_match_distance_m', 'spill_match_distance_m']

# Drop the columns from the DataFrame
combined_gdf = combined_gdf.drop(columns=columns_to_remove)

In [10]:
# Reorder df
new_order = ['unique_id', 'operator_number', 'FLOWLINEID', 'LOCATION_ID', 'Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid',
             'Material', 'Diam_in', 'Length_ft', 'MAXOPPRESSURE', 'line_age_yr', 'CONSTRUCTDATE', 'incident_date','Root Cause Type','geometry', 'risk']
combined_gdf = combined_gdf[new_order]
combined_gdf

,unique_id,operator_number,FLOWLINEID,LOCATION_ID,Status,FLOWLINEACTION,LOCATIONTYPE,Fluid,Material,Diam_in,Length_ft,MAXOPPRESSURE,line_age_yr,CONSTRUCTDATE,incident_date,Root Cause Type,geometry,risk
0,16962,10110,470450.0,470449.0,Active,Pre-Abandonment Notice,Production Facilities,Oil,Steel,2.0,542.71,NaN,52.780287,1972-08-07,2018-01-23 00:00:00,Unknown,"LINESTRING (-104.47065 39.8447, -104.47245 39....",1
1,8053,69175,471990.0,471975.0,Partial Removed see comment,Registration,Production Facilities,None,None,NaN,353.54,NaN,32.388775,1992-12-28,2018-02-08 00:00:00,Unknown,"LINESTRING (-104.67983 40.3862, -104.67857 40....",1
2,145049,47120,462980.0,460727.0,REMOVED,Abandonment,Production Facilities,PRODUCED WATER,None,1.0,162.94,NaN,19.195072,2006-03-09,2018-03-09 00:00:00,Unknown,"LINESTRING (-104.63096 40.1376, -104.63093 40....",1
3,34293,100322,457928.0,457851.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,2000.31,NaN,38.165640,1987-03-20,2018-03-15 00:00:00,Unknown,"LINESTRING (-104.60942 40.32153, -104.60267 40...",1
4,95145,100322,455852.0,455178.0,Active,Registration,Production Facilities,Multiphase,Carbon Steel,2.0,852.25,NaN,28.194387,1997-03-09,2018-03-22 00:00:00,Unknown,"LINESTRING (-104.7565 40.48315, -104.75445 40....",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163132,159140,10456,476084.0,476062.0,Out of Service,None,Well Site,Produced Water,Steel,8.0,1096.64,700.0,19.964408,2005-06-01,None,None,"LINESTRING (-108.05 39.39961, -108.05252 39.39...",0
163133,159141,10456,476810.0,335102.0,Abandoned,Registration,Well Site,Produced Water,High-density polyethylene (HDPE),12.0,4.69,700.0,20.544832,2004-11-01,None,None,"LINESTRING (-107.67276 39.47018, -107.67276 39...",0
163134,159142,10559,479239.0,320870.0,Active,Registration,Well Site,Produced Water,Carbon Steel,2.0,68.48,80.0,19.980835,2005-05-26,None,None,"LINESTRING (-107.4389 37.03008, -107.43886 37....",0
163135,159143,96850,464813.0,323946.0,Active,Registration,Production Facilities,Produced Water,FlexPipe,4.0,1058.70,NaN,16.323066,2009-01-21,None,None,"LINESTRING (-107.93466 39.47953, -107.93197 39...",0


### Consolidate variables uniformly

In [11]:
# List of specific columns for which to print unique values
columns_to_check = ['Status', 'FLOWLINEACTION', 'LOCATIONTYPE', 'Fluid', 'Material']

# Create a dictionary to store unique values for each column
unique_values_dict = {}

# Loop through the specified columns and store unique values in the dictionary
for column in columns_to_check:
    if column in combined_gdf.columns:  # Check if the column exists in the DataFrame
        unique_values = combined_gdf[column].unique().tolist()  # Convert numpy array to list
        unique_values_dict[column] = unique_values
    else:
        unique_values_dict[column] = "Column not found in DataFrame."

# Print each column's unique values list
for column, values in unique_values_dict.items():
    print(f"Unique values in {column}: {values}")

Unique values in Status: ['Active', 'Partial Removed see comment', 'REMOVED', 'Abandoned', 'ACTIVE', 'Out of Service', None, 'Out-of-Service', 'ABANDONED', 'Abandon', 'Removed', 'abandoned', 'ABIP', 'Out Of Service', 'New Construction', 'INACTIVE', 'Nitrogen', 'Inactive', 'Pre-Commissioned', 'PreCommission', 'ABiP', 'InActive', 'SI', 'TA', 'PA', 'Flushed and capped', 'Discontinued', 'OUT OF SERVICE', 'OOS', 'OutofService', 'Pre Abandonment', 'PreAbandonment', 'Pre-Abandoned', 'Injection', 'REMOVED DUPLICATE', 'Abandoned in Place', 'Pre-commissioned', 'Abadnon', 'Actuve', 'Plugged', 'Avtive', 'AB', 'Pre-Abandonment', 'OOSLAT', 'Turned into Air Line', 'Future', 'active', 'Actve', 'shut in', '0', 'Out of service', 'Status', 'Flow filled and capped', 'Actove', 'Injecting', 'Shut in', '3rd Party Line', 'unk', 'AIP', 'Planned', 'Flowline', 'IDLE', 'Abandonded']
Unique values in FLOWLINEACTION: ['Pre-Abandonment Notice', 'Registration', 'Abandonment', None, 'Abandonment Verification', 'Out of

In [12]:
status_mapping = {
    'Active': 'Active', 'ACTIVE': 'Active', 'Actove': 'Active', 'Avtive': 'Active', 'Actve': 'Active', 'active': 'Active',
    'Out of Service': 'Out of Service', 'OOS': 'Out of Service', 'OutofService': 'Out of Service', 'Out-of-Service': 'Out of Service', 'Out Of Service': 'Out of Service', 'Out of service': 'Out of Service',
    'Abandoned': 'Abandoned', 'abandoned': 'Abandoned', 'Abandoned in Place': 'Abandoned', 'ABANDONED': 'Abandoned', 'Abandon': 'Abandoned','Abadnon': 'Abandoned','TA': 'Abandoned',
    'Inactive': 'Inactive', 'InActive': 'Inactive', 'INACTIVE': 'Inactive',
    'PA': 'Pending Analysis', 'ABiP':'Pending Analysis', 'Shut in': 'Shut In', 'shut in': 'Shut In','SI':'Shut In',
    'Status': 'Unknown', 'Future': 'Future',
    'REMOVED': 'Removed',
    'Pre Abandonment':'Pre-Abandonment', 'PreAbandonment': 'Pre-Abandonment'
}
combined_gdf['Status'] = combined_gdf['Status'].replace(status_mapping)
print(combined_gdf['Status'].unique())


flowlineaction_mapping = {
    'Out of Service': 'Out of Service', 'Removed From Service': 'Out of Service',
    'Pre-Abandonment Notice': 'Pre-Abandonment Notice',
    'Abandonment Verification': 'Abandonment',
    'Realignment': 'Realignment',
    'Registration': 'Registration',
    'Abandonment': 'Abandonment'
}
combined_gdf['FLOWLINEACTION'] = combined_gdf['FLOWLINEACTION'].replace(flowlineaction_mapping)
print(combined_gdf['FLOWLINEACTION'].unique())


locationtype_mapping = {
    'Production Facilities': 'Production Facilities', 'Well Site': 'Well Site', 'Manifold': 'Manifold',
    'Compressor Station': 'Compressor Station', 'Gathering Line': 'Gathering Line',
    'Crude Oil Transfer Line': 'Crude Oil Transfer Line', 'Produced Water Transfer System': 'Produced Water Transfer System'
}
combined_gdf['LOCATIONTYPE'] = combined_gdf['LOCATIONTYPE'].replace(locationtype_mapping)
print(combined_gdf['LOCATIONTYPE'].unique())

['Active' 'Partial Removed see comment' 'Removed' 'Abandoned'
 'Out of Service' None 'ABIP' 'New Construction' 'Inactive' 'Nitrogen'
 'Pre-Commissioned' 'PreCommission' 'Pending Analysis' 'Shut In'
 'Flushed and capped' 'Discontinued' 'OUT OF SERVICE' 'Pre-Abandonment'
 'Pre-Abandoned' 'Injection' 'REMOVED DUPLICATE' 'Pre-commissioned'
 'Actuve' 'Plugged' 'AB' 'OOSLAT' 'Turned into Air Line' 'Future' '0'
 'Unknown' 'Flow filled and capped' 'Injecting' '3rd Party Line' 'unk'
 'AIP' 'Planned' 'Flowline' 'IDLE' 'Abandonded']
['Pre-Abandonment Notice' 'Registration' 'Abandonment' None
 'Out of Service' 'Realignment']
['Production Facilities' 'Manifold' 'Well Site' 'Pit' 'Compressor Station'
 'Gathering Line' 'Produced Water Transfer System'
 'Crude Oil Transfer Line']


In [13]:
# Update the 'Fluid' column normalization and mapping in one comprehensive block
combined_gdf['Fluid'] = combined_gdf['Fluid'].str.strip().str.title().replace({
    'Natual Gas': 'Natural Gas',  # Correct common misspelling
    'Natural Gas Production': 'Natural Gas',
    'Co2': 'Co2/Produced Water',  # Assuming Co2 implies the mixed type
    'C02/Prod Water': 'Co2/Produced Water',
    'Co2/Prod Water': 'Co2/Produced Water',
    'Co2Produced Water': 'Co2/Produced Water',
    'Co2/Produced Wtaer': 'Co2/Produced Water',
    'Gas': 'Natural Gas',  # Assuming general Gas to be Natural Gas
    'Gas, Oil And Water': 'Full Well Stream',
    'Oil': 'Crude Oil',  # Assuming Oil to be categorized as Crude Oil
    'Crude Oil': 'Crude Oil',
    'Crude Oil Emulsion': 'Crude Oil Emulsion',
    'Emulsion': 'Crude Oil Emulsion',  # To unify with Crude Oil Emulsion
    'Crude Oil Emmulsion, Water And Oil': 'Crude Oil Emulsion',
    'Crude Oil And Water Emulsion': 'Crude Oil Emulsion',
    'Oil Water Emulsion': 'Crude Oil Emulsion',
    'Oil/Water': 'Crude Oil Emulsion',
    'Oil Water': 'Crude Oil Emulsion',
    'Oil And Water': 'Crude Oil Emulsion',
    'Oil /Water/Gas': 'Full Well Stream',
    'Oil/Gas/Water': 'Full Well Stream',
    'Oil, Gas, Water': 'Full Well Stream',
    '3 Phase': 'Multiphase',  # Assumed to mean the same
    'Multiphase': 'Multiphase',
    'Multi-Phase': 'Multiphase',
    'Mulitphase': 'Multiphase',
    'Multi Phase': 'Multiphase',
    'Mulit Phase': 'Multiphase',
    'Multi-Phase\xa0': 'Multiphase',  # Non-breaking space issue
    'Injection Produced Water': 'Produced Water',
    'Produced Water': 'Produced Water',
    'Water': 'Produced Water',
    'Saltwater': 'Produced Water',
    'Condensate': 'Condensate',  # Depending on your classification needs
    'Liquid': 'Other',
    'Liquids (Wtr/Cond)': 'Other', # General term, assuming to categorize broadly
    'Unprocessed Production Fluids': 'Other',
    'Production Fluids': 'Other',
    'Produced Fluids': 'Other',
    'Full Well Stream': 'Full Well Stream',  # Assuming similar handling
    'Other': 'Other',
    'Gas,  Oil And Water' : 'Full Well Stream',
    'Natural Gas Lift': 'Natural Gas',
    'Natuarl Gas': 'Natural Gas',
    'Natural Gas High Pressure': 'Natural Gas',
    'Natural Gas Supply':'Natural Gas',
    'Crude Oill Emulsion': 'Crude Oil Emulsion',
    'Unk': 'Unknown', 
    'Poly':'Polymer fluids'
})

# Review the changes to ensure the mappings are applied correctly
print(combined_gdf['Fluid'].unique())

['Crude Oil' None 'Produced Water' 'Multiphase' 'Natural Gas'
 'Co2/Produced Water' 'Crude Oil Emulsion' 'Other' 'Condensate'
 'Full Well Stream' 'Service Line' '(Other) Treated Produced Water'
 'Fresh Water' 'Natural Gas/Condensate/Produced Water' 'Supply Gas'
 'Mutliphase' 'Vent' 'Co2 Produced Water' 'Air' 'Polymer fluids' 'Fluid'
 'Oil, Water, Gas' 'Fuel Gas' 'Other- Nitrogen/ Natural Gas'
 'Crude Oil / Produced Water' 'Crude Oil Unprocessed' 'Natural Gas Sales'
 'Unknown' 'Steel' 'Co2/Produced'
 'Waste Water/Produced Water/Formation Water' 'Cox V2' 'Oil-Gas-Water'
 'Produced  Water' 'Produced/Waste Water']


In [14]:
combined_gdf['Material'] = combined_gdf['Material'].str.strip().str.title().replace({
    'Fiberglass': 'Fiberglass', 
    'Fibergalss': 'Fiberglass', 
    'Fiberspar': 'Fiberglass', 
    'Fiber Glass': 'Fiberglass',
    'Carbon Steel': 'Carbon Steel', 
    'Carbonsteel': 'Carbon Steel',
    'Carbon Steel Sch 80': 'Carbon Steel',
    'Carbon Steel Sch 80': 'Carbon Steel',
    'Carbon Steel - Hdpe': 'Carbon Steel/HDPE', 
    'Carbon Steel, Hdpe,Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel, Hdpe, Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe/Stainless': 'Carbon Steel/HDPE/Stainless Steel', 
    'Carbon Steel/Hdpe': 'Carbon Steel/HDPE', 
    'Satinless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Stainless/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Steel': 'Steel', 
    'Lined Steel': 'Steel', 
    'Coated Steel': 'Steel', 
    'Flexsteel': 'Steel', 
    'Flexpipe' : 'Steel',
    'Fiber Glass And Carbon Steel': 'Fiberglass/Carbon Steel', 
    'Fiberglass And Hdpe': 'Fiberglass/HDPE',
    'Hdpe': 'HDPE', 
    'Hdpe Poly': 'HDPE', 
    'Composite Hdpe': 'HDPE',
    'Hdpe/Steel': 'HDPE/Steel', 
    'Hdpe Lined Steel': 'HDPE/Steel',
     'Hdpe/Steel, Flexsteel': 'HDPE/Steel',
    'Poly': 'Polycarbonate', 
    'Polyline': 'Polycarbonate', 
    'Poly & Steel': 'Polycarbonate/Steel', 
    'Steel/Poly': 'Polycarbonate/Steel',
    'Poly/Steel': 'Polycarbonate/Steel',
    'Polycarbonate': 'Polycarbonate', 
    'Polycarbonate/Steel': 'Polycarbonate/Steel',
    'Pvc': 'PVC', 
    'Flexspar': 'Fiberglass', 
    'Stainless': 'Steel', 
    'Stainless/Carbon Steel/Hdpe': 'Carbon Steel/HDPE/Stainless Steel',
    'Carbon Steel/Hdpe/Stainless Steel': 'Carbon Steel/HDPE/Stainless Steel',
    'Unknown': 'Unknown', 
    'Other': 'Other', 
    'Other (Poly)': 'Polycarbonate', 
    'Sdr7 Polyethelyne': 'Polyethylene', 
    'Sdr 11 Poly Pipe': 'Polyethylene', 
    'Sdr 11 Poly': 'Polyethylene', 
    'Poly Pipe': 'Polyethylene', 
    'Sdr_Poly': 'Polyethylene',
    'Poly': 'Polyethylene',
    'Poly Sdr 7': 'Polypropylene',
    'Poly Sdr-7': 'Polypropylene',
    'Duplex': 'Duplex', 
    'Fplp': 'Other', 
    'Flowline': 'Other',
    'Flex Steel': 'Steel',
    'Other (Flex Steel)': 'Steel',
    'Fiberglass And Carbon Steel': 'Carbon Steel/Fiberglass', 
    'Stainless Steel': 'Steel',
    'HDPE Lined Steel': 'HDPE/Steel',
    'Fiberglass/Hdpe': 'Fiberglass/HDPE',
    'Unk': 'Unknown', 'Other (Unknown)': 'Unknown', 'Other': 'Unknown',
})

# Verify the changes by printing the unique values in the 'Material' column
print(combined_gdf['Material'].unique())

['Steel' None 'Carbon Steel' 'HDPE' 'Fiberglass' 'Other (Stainless Steel)'
 'Flex Pipe' 'Polyethylene' 'Other' 'Polycarbonate'
 'Carbon Steel/HDPE/Stainless Steel' 'High-Density Polyethylene (Hdpe)'
 'Core Linepipe' 'PVC' 'Unknown' 'Sdr 7 Poly' 'Fiberglass Sleaved W/ Hdpe'
 'Hdpe Sdr11' 'Plastic' 'Carbon  Steel' 'Other (Flex Pipe)' 'Hdpi Poly'
 'Duplex' 'Stainless Steel/Carbon Steel/Hdpe' 'Co2/Produced Water'
 'Other (Please Specify)' 'Sdr-11 Poly' 'Polycarbonate/Steel'
 'Carbon Steel-Hdpe' 'Other (Flexsteel)' 'Carbon Steel And Hdpe' 'Sdr-11'
 'Aluminum' 'Stainless/Carbonsteel/Hdpe' 'Shawcor Fp150'
 'Hdpe Poly Sdr 11' 'Carbon Steel/HDPE' 'HDPE/Steel'
 'Carbon Steel/Fiberglass' 'Stainless/ Carbon Steel/Hdpe' 'Polypropylene'
 '0' 'Fiberglass/HDPE' 'Sdr' 'Oil' 'Fiberglass & Fiberspar' 'Gas'
 'Hdpe Line Sdr 7' 'Stainless/Carbon Steel/Hspe'
 'Carbon Steel Mixed With Fiberglass' 'Stainless/Carbon Steel/ Hdpe'
 'Zaplock' 'Other (Hdpe And Tubing)' 'Hdpe Sdr7' 'Hdpe/Fiberglass'
 'Poly Sleeved S

In [15]:
# Define the mapping dictionary for the 'Root Cause Type' column
root_cause_mapping = {
    'Corrosion': 'Corrosion', 
    'Unknown': 'Unknown', 
    'Incorrect Operation': 'Incorrect Operation', 
    'Equipment Failure': 'Equipment Failure', 
    'Equipment failure': 'Equipment Failure',  
    'Other Outside Force Damage': 'Other Outside Force Damage', 
    'Natural Force Damage': 'Natural Force Damage', 
    'Pipe, Weld, or Joint Failure': 'Pipe, Weld, or Joint Failure', 
    'Pipe, Weld Joint Failure': 'Pipe, Weld, or Joint Failure',
    'Excavation Damage': 'Excavation Damage', 
    'Other Outside Force': 'Other Outside Force Damage',  # Mapping to a more standardized category
    'Pipe, Weld, Joint Failure': 'Pipe, Weld, or Joint Failure'  # Ensuring consistency in naming
}

# Apply the mapping to the 'Root Cause Type' column
combined_gdf['Root Cause Type'] = combined_gdf['Root Cause Type'].replace(root_cause_mapping)
print(combined_gdf['Root Cause Type'].unique())

['Unknown' 'Corrosion' 'Equipment Failure' 'Incorrect Operations'
 'Pipe, Weld, or Joint Failure' 'Other Outside Force Damage'
 'Natural Force Damage' 'Excavation Damage' None]


In [16]:
combined_gdf.columns = [
    'unique_id',
    'operator_number',  # Changed from OPERATOR_NUM to operator_number
    'flowline_id',       # Changed from FLOWLINEID to flowline_id
    'location_id',       # Changed from LOCATION_ID to location_id
    'status',            # Already appropriately named but changed to lowercase
    'flowline_action',   # Changed from FLOWLINEACTION to flowline_action
    'location_type',     # Changed from LOCATIONTYPE to location_type
    'fluid',             # Already appropriately named but changed to lowercase
    'material',          # Already appropriately named but changed to lowercase
    'diameter_in',       # Changed from Diam_in to diameter_in
    'length_ft',         # Changed from Length_ft to length_ft
    'max_operating_pressure',
    'line_age_yr',
    'construct_date',
    'spill_date',
    'root_cause',
    'geometry',
    'risk',
    ]

# Print new column names to verify
print(combined_gdf.columns.tolist())

['unique_id', 'operator_number', 'flowline_id', 'location_id', 'status', 'flowline_action', 'location_type', 'fluid', 'material', 'diameter_in', 'length_ft', 'max_operating_pressure', 'line_age_yr', 'construct_date', 'spill_date', 'root_cause', 'geometry', 'risk']


In [17]:
combined_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 163137 entries, 0 to 163136
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   unique_id               163137 non-null  int32         
 1   operator_number         163137 non-null  int32         
 2   flowline_id             163023 non-null  float64       
 3   location_id             163113 non-null  float64       
 4   status                  129183 non-null  object        
 5   flowline_action         153765 non-null  object        
 6   location_type           163137 non-null  object        
 7   fluid                   160923 non-null  object        
 8   material                157338 non-null  object        
 9   diameter_in             105790 non-null  float64       
 10  length_ft               163137 non-null  float64       
 11  max_operating_pressure  32133 non-null   float64       
 12  line_age_yr           

In [18]:
combined_gdf.shape

(163137, 18)

In [19]:
# Remove rows where 'max_operating_pressure' is NaN and 'risk' is 0
combined_gdf = combined_gdf[~((combined_gdf['max_operating_pressure'].isna()) & (combined_gdf['risk'] == 0))]

In [20]:
combined_gdf.shape

(32562, 18)

In [21]:
# If you want the count of each risk class (e.g. 0 vs 1):
counts = combined_gdf['risk'].value_counts()
print("Counts per risk class:\n", counts)

Counts per risk class:
 risk
0    31728
1      834
Name: count, dtype: int64


In [22]:
# Select the relevant numeric columns for KNN Imputation
columns_for_imputation = ['max_operating_pressure', 'diameter_in', 'length_ft', 'line_age_yr', 'material', 'fluid']  # Add other relevant columns as needed

# Creating a copy of the data to prevent modifying the original directly
df_for_imputation = combined_gdf[columns_for_imputation].copy()

# Convert the 'material' column using Label Encoding
le = LabelEncoder()
df_for_imputation['material_encoded'] = le.fit_transform(df_for_imputation['material'])
df_for_imputation['fluid_encoded'] = le.fit_transform(df_for_imputation['fluid'])

# Drop the original 'material' and 'fluid' columns
df_for_imputation = df_for_imputation.drop(columns=['material', 'fluid'])

# Initialize the KNN Imputer
imputer = KNNImputer(n_neighbors=5)

# Fit and transform the data to impute missing values
df_imputed = imputer.fit_transform(df_for_imputation)

# Replace the original max_operating_pressure with the imputed values
combined_gdf['max_operating_pressure'] = df_imputed[:, 0]

/Users/ichittumuri/Desktop/MINES/COGCC-Risk-Analysis/ecmc_env/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [23]:
# Function to count only NaN values, excluding None
def count_only_nan(series):
    return series.apply(lambda x: 1 if isinstance(x, float) and np.isnan(x) else 0).sum()

# Apply the function to each column to count NaNs, not None
na_columns = combined_gdf.apply(count_only_nan)

# Filter columns with NaN values
columns_with_only_nan = na_columns[na_columns > 0]

# Print the results
print("Columns with NaN values (excluding None) and their counts:")
print(columns_with_only_nan)


Columns with NaN values (excluding None) and their counts:
flowline_id      49
location_id      15
diameter_in    1104
dtype: int64


In [24]:
# Identify columns with NaN values
columns_with_na = combined_gdf.columns[combined_gdf.isna().any()]

# Create a dictionary to store the counts of NaNs with risk of 1 for each column
na_with_risk_1 = {}

# Iterate over each column with NaN values and count rows with risk = 1
for column in columns_with_na:
    count = combined_gdf[combined_gdf[column].isna() & (combined_gdf['risk'] == 1)].shape[0]
    na_with_risk_1[column] = count

# Print the counts of NaNs with risk of 1 for each column
print("Number of NaNs with risk of 1 in each column:")
print(na_with_risk_1)

Number of NaNs with risk of 1 in each column:
{'flowline_id': 0, 'location_id': 0, 'status': 48, 'flowline_action': 233, 'fluid': 28, 'material': 67, 'diameter_in': 86, 'spill_date': 0, 'root_cause': 0}


In [25]:
# If you want the count of each risk class (e.g. 0 vs 1):
counts = combined_gdf['risk'].value_counts()
print("Counts per risk class:\n", counts)

Counts per risk class:
 risk
0    31728
1      834
Name: count, dtype: int64


In [26]:
counts = {
  'flowline_id': 0,
  'location_id': 0,
  'status': 48,
  'flowline_action': 233,
  'fluid': 28,
  'material': 67,
  'diameter_in': 86,
  'spill_date': 0,
  'root_cause': 0
}

to_fill = ['status','flowline_action','fluid','material']

for col in to_fill:
    combined_gdf.loc[
        (combined_gdf['risk'] == 1) & combined_gdf[col].isna(),
        col
    ] = 'None'

In [27]:
combined_gdf = combined_gdf.dropna(subset=['flowline_id', 'location_id', 'diameter_in'])

In [28]:
# make variables integers
combined_gdf['diameter_in'] = combined_gdf['diameter_in'].astype(int)
combined_gdf['length_ft'] = combined_gdf['length_ft'].astype(int)
combined_gdf['max_operating_pressure'] = combined_gdf['max_operating_pressure'].astype(int)
combined_gdf['line_age_yr'] = combined_gdf['line_age_yr'].astype(int)

In [29]:
# Calculate the total number of rows
total_rows = combined_gdf.shape[0]

# Count rows where risk is 1
risk_1_count = combined_gdf[combined_gdf['risk'] == 1].shape[0]

# Count rows where risk is 0
risk_0_count = combined_gdf[combined_gdf['risk'] == 0].shape[0]

# Print the results
print(f"Total number of rows: {total_rows}")
print(f"Total number of rows with risk = 1: {risk_1_count}")
print(f"Total number of rows with risk = 0: {risk_0_count}")

Total number of rows: 31418
Total number of rows with risk = 1: 748
Total number of rows with risk = 0: 30670


# Download Cleaned Data

In [30]:
combined_gdf.to_file("cleaned_gdf.geojson", driver='GeoJSON')

In [31]:
combined_gdf_operator_mapping.to_csv('operator_mapping.csv', index=False)